In [1]:
import numpy as np
from Bio import SeqIO
import tables
#from bert_serving.client import BertClient
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tokenization 
celline = "IMR90"
file_pre  = 'aug_50/'+celline
import pickle

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def one_hot_encode(sequences):
    sequence_length = len(sequences[0])
    integer_type = np.int32
    integer_array = LabelEncoder().fit(np.array(('ACGT',)).view(integer_type)).transform(
        sequences.view(integer_type)).reshape(len(sequences), sequence_length)
    
    
    one_hot_encoding = OneHotEncoder(handle_unknown='ignore',
        sparse=False,dtype=integer_type).fit_transform(integer_array)
    print(one_hot_encoding.shape)
    return one_hot_encoding.reshape(
        len(sequences),sequence_length, 4)


def input_sequences(efile, pfile, labelfile):
                        seq_e = []
                        seq_p = []
                        enhancer_sequences = SeqIO.parse(open(efile), 'fasta')
                        promoter_sequences = SeqIO.parse(open(pfile), 'fasta')
                        for enhancer in enhancer_sequences:
                            name, sequence = enhancer.id, str(enhancer.seq)
                            seq_e.append(sequence)
    
                        for promoter in promoter_sequences:
                            name, sequence = promoter.id, str(promoter.seq)
                            seq_p.append(sequence)
    
                        seq_e = np.array(seq_e)
                        seq_p = np.array(seq_p)
                        X_e = one_hot_encode(seq_e)
                        X_p = one_hot_encode(seq_p)

    #split to seq_e_p, seq_e_n, seq_p_p and seq_p_n based on the labels
                        num_pos = 0
                        num_neg = 0    
                        with open(labelfile) as f:
                            for line in f:
                                if line.rstrip() == '1':
                                    num_pos = num_pos+1
                                else:
                                    num_neg = num_neg+1
                        label = np.concatenate([np.ones([num_pos,1]), np.zeros([num_neg,1])])
                        return X_e[0:num_pos], X_e[num_pos:], X_p[0:num_pos], X_p[num_pos:],label




In [2]:
seq_ep, seq_en, seq_pp, seq_pn,label= input_sequences(efile = file_pre+'_enhancer.fasta', 
                                                 pfile = file_pre+'_promoter.fasta', 
                                                 labelfile = file_pre+'_label.txt')
print(seq_ep.shape,seq_pp.shape)
np.savez(celline+'one_hot_train.npz',enhancer_pos=seq_ep,enhancer_neg=seq_en,promoter_pos=seq_pp,promoter_neg=seq_pn,label=label)
seq_ep_t, seq_en_t, seq_pp_t, seq_pn_t,label= input_sequences(efile =file_pre+'_enhancer_test.fasta', 
                                                         pfile = file_pre+'_promoter_test.fasta', 
                                                         labelfile = file_pre+'_label_test.txt')
np.savez(celline+'one_hot_test.npz',enhancer_pos=seq_ep,enhancer_neg=seq_en,promoter_pos=seq_pp,promoter_neg=seq_pn,label=label)

NameError: name 'input_sequences' is not defined

['ATCGC']
(1,)
